# 1.) Imports

In [ ]:
import torch
import numpy as np
import scipy.ndimage
import random
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pandas as pd

import torch
import random

In [ ]:
class BrainAgeTrainer:
    def __init__(self, model, train_loader, val_loader, criterion, optimizer, device,
                 scheduler=None, augment=True, early_stopping_patience=3, use_weighted_loss=False):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.criterion = criterion
        self.optimizer = optimizer
        self.scheduler = scheduler
        self.device = device
        self.augment = augment
        self.use_weighted_loss = use_weighted_loss

        self.history = {'train_loss': [], 'val_loss': []}
        self.train_preds = []
        self.train_targets = []
        self.val_preds = []
        self.val_targets = []
        self.early_stopping_patience = early_stopping_patience
        self.best_val_loss = float('inf')
        self.epochs_without_improvement = 0
        self.best_model_state = None

    def train(self, epochs=10, track_predictions=False):
        self.model.to(self.device)

        for epoch in range(epochs):
            print(f"\n🔁 Epoch {epoch+1}/{epochs}")
            self.model.train()
            train_loss = 0.0

            if track_predictions:
                self.train_preds.clear()
                self.train_targets.clear()

            for i, batch in enumerate(self.train_loader):
                if self.use_weighted_loss:
                    images, ages, weights = batch
                    weights = weights.to(self.device)
                else:
                    images, ages = batch
                    weights = None

                images = images.to(self.device)
                ages = ages.to(self.device).view(-1)

                if self.augment:
                    images = torch.stack([self.augment_volume(img) for img in images])

                self.optimizer.zero_grad()
                outputs = self.model(images)

                if self.use_weighted_loss:
                    loss = torch.mean(weights * torch.abs(outputs - ages))
                else:
                    loss = self.criterion(outputs, ages)

                if hasattr(self.model, 'age_scale') and hasattr(self.model, 'age_bias'):
                    reg_lambda = 0.01
                    reg_term = reg_lambda * (self.model.age_scale**2 + self.model.age_bias**2)
                    loss = loss + reg_term

                loss.backward()
                self.optimizer.step()

                train_loss += loss.item() * images.size(0)

                if track_predictions:
                    self.train_preds.extend(outputs.detach().cpu().numpy().reshape(-1))
                    self.train_targets.extend(ages.cpu().numpy().reshape(-1))

            train_loss /= len(self.train_loader.dataset)
            val_loss = self._validate(track_predictions)

            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)

            print(f"📉 Train Loss: {train_loss:.4f} | 🧪 Val Loss: {val_loss:.4f}")

            if hasattr(self.model, 'age_scale') and hasattr(self.model, 'age_bias'):
                scale = self.model.age_scale.item()
                bias = self.model.age_bias.item()
                print(f"🧮 Calibration → age_scale: {scale:.4f}, age_bias: {bias:.4f}")

            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.epochs_without_improvement = 0
                self.best_model_state = self.model.state_dict()
                print("💾 Best model updated")
            else:
                self.epochs_without_improvement += 1
                if self.epochs_without_improvement >= self.early_stopping_patience:
                    print("🛑 Early stopping triggered")
                    break

            if self.scheduler:
                self.scheduler.step(val_loss)
                current_lr = self.optimizer.param_groups[0]['lr']
                print(f"📉 Current Learning Rate: {current_lr:.2e}")

        if self.best_model_state is not None:
            self.model.load_state_dict(self.best_model_state)
            print("✅ Restored best model weights")

    def _validate(self, track_predictions):
        self.model.eval()
        val_loss = 0.0

        if track_predictions:
            self.val_preds.clear()
            self.val_targets.clear()

        with torch.no_grad():
            for images, ages in self.val_loader:
                images, ages = images.to(self.device), ages.to(self.device)
                outputs = self.model(images)
                loss = self.criterion(outputs, ages)
                val_loss += loss.item() * images.size(0)

                if track_predictions:
                    self.val_preds.extend(outputs.cpu().numpy().reshape(-1))
                    self.val_targets.extend(ages.cpu().numpy().reshape(-1))

        return val_loss / len(self.val_loader.dataset)

    def get_history(self):
        return self.history

    def get_predictions(self):
        return {
            'train': (self.train_preds, self.train_targets),
            'val': (self.val_preds, self.val_targets)
        }

    def augment_volume(self, volume, flip_prob=0.5, noise_std=0.002):
        volume = volume.clone()

        if random.random() < flip_prob:
            volume = torch.flip(volume, dims=[1])
        if random.random() < flip_prob:
            volume = torch.flip(volume, dims=[2])
        if random.random() < flip_prob:
            volume = torch.flip(volume, dims=[3])

        volume += torch.randn_like(volume) * noise_std

        scale = random.uniform(0.95, 1.05)
        shift = random.uniform(-0.05, 0.05)
        volume = volume * scale + shift

        return volume.to(self.device)

def compute_age_weights(df, bins=8, normalize=True):
    df = df.copy()
    df['age_bin'] = pd.cut(df['Age'], bins=bins)
    bin_counts = df['age_bin'].value_counts().sort_index()
    weights = 1.0 / bin_counts

    if normalize:
        weights /= weights.max()

    return dict(zip(bin_counts.index, weights))

def compute_balanced_age_weights(df, bins=8, normalize=True, custom_boost=None):
    df = df.copy()
    df['age_bin'] = pd.qcut(df['Age'], q=bins, duplicates='drop')
    bin_counts = df['age_bin'].value_counts().sort_index()
    weights = 1.0 / bin_counts.astype(float)

    if custom_boost is not None:
        assert len(custom_boost) == len(weights), f"custom_boost must match number of bins ({len(weights)})"
        weights *= np.array(custom_boost)

    if normalize:
        weights /= weights.max()

    print("\n📊 Age Bin Weights:")
    for bin_interval, weight in zip(bin_counts.index, weights):
        print(f"{bin_interval}: {weight:.4f}")

    return dict(zip(bin_counts.index, weights))